<a href="https://colab.research.google.com/github/AimeePatience/AimeePatience/blob/main/Multi_Agent_Review_Board.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# README

# Multi agent review board notebook using "mmlu" data (Source code has "geography", "gsm", "math", and "mmlu" data)
# Source code: https://github.com/composable-models/llm_multiagent_debate

# CONFIGURE parameters like which models to use and agent configs using fourth code block/cell.

In [2]:
# Install depencies and authenticate with hugging face to access gated models
!pip install transformers accelerate -q

# Authenticate with Hugging Face
from huggingface_hub import login
login()  # You'll need a HF token

# Task
Modify the code in cell `GH-l3bNs-PgQ` to define distinct `model_id`s and create a separate `pipeline` for each, then update `agent_configs` to assign these unique pipelines to each agent.

## Modify Model Configuration

### Subtask:
Update the code in cell `GH-l3bNs-PgQ` to define multiple `model_id`s, create a separate `pipeline` for each model, and then configure `agent_configs` to use these distinct pipelines. This will ensure each agent utilizes a different model as requested.


**Reasoning**:
The subtask requires modifying cell `GH-l3bNs-PgQ` to use multiple distinct models and pipelines for the agents. I will update the code to define three different `model_id`s, create a separate `pipeline` for each, and then assign these pipelines to the `agent_configs` entries, ensuring each agent uses a different model.



In [7]:
from glob import glob
import pandas as pd
import json
import time
import random


In [8]:
# Load MMLU data and store data frames
!curl "https://people.eecs.berkeley.edu/~hendrycks/data.tar" | tar -xvf -

tasks = glob("./data/test/*.csv")
dfs = [pd.read_csv(task) for task in tasks]

len(dfs)


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0data/
data/README.txt
data/val/
data/val/high_school_geography_val.csv
data/val/astronomy_val.csv
  0  158M    0 14992    0     0  15027      0  3:04:19 --:--:--  3:04:19 15022data/val/conceptual_physics_val.csv
data/val/medical_genetics_val.csv
data/val/high_school_computer_science_val.csv
data/val/business_ethics_val.csv
data/val/anatomy_val.csv
data/val/global_facts_val.csv
data/val/philosophy_val.csv
data/val/high_school_government_and_politics_val.csv
data/val/college_physics_val.csv
data/val/miscellaneous_val.csv
data/val/machine_learning_val.csv
data/val/high_school_mathematics_val.csv
data/val/high_school_biology_val.csv
data/val/abstract_algebra_val.csv
data/val/college_medicine_val.csv
data/val/clinical_knowledge_val.csv
data/val/electrical_eng

57

In [9]:
def construct_message(agents, question, idx):
    if len(agents) == 0:
        return {"role": "user", "content": "Can you double check that your answer is correct. Put your final answer in the form (X) at the end of your response."}

    prefix_string = "These are the solutions to the problem from other agents: "

    for agent in agents:
        agent_response = agent[idx]["content"]
        response = "\n\n One agent solution: ```{}```".format(agent_response)

        prefix_string = prefix_string + response

    prefix_string = prefix_string + """\n\n Using the reasoning from other agents as additional advice, can you give an updated answer? Examine your solution and that other agents step by step. Put your answer in the form (X) at the end of your response.""".format(question)
    return {"role": "user", "content": prefix_string}


def construct_assistant_message(completion):
    # content = completion["choices"][0]["message"]["content"]
    content = completion[0]["generated_text"][-1]["content"]
    return {"role": "assistant", "content": content}


def generate_answer(agent_config, agent_context):
    # try:

    #     completion = openai.ChatCompletion.create(
    #               model="gpt-3.5-turbo-0301",
    #               messages=answer_context,
    #               n=1)
    # except:
    #     print("retrying due to an error......")
    #     time.sleep(20)
    #     return generate_answer(answer_context)
    completion = agent_config["pipe"](
      agent_context,
      max_new_tokens=1024,
      do_sample=True,
      temperature=agent_config["temp"],
      top_p=0.9,
    )


    return completion


def parse_question_answer(df, ix):
    question = df.iloc[ix, 0]
    a = df.iloc[ix, 1]
    b = df.iloc[ix, 2]
    c = df.iloc[ix, 3]
    d = df.iloc[ix, 4]

    question = "Can you answer the following question as accurately as possible? {}: A) {}, B) {}, C) {}, D) {} Explain your answer, putting the answer in the form (X) at the end of your response.".format(question, a, b, c, d)

    answer = df.iloc[ix, 5]

    return question, answer



In [ ]:
random.seed(0)
response_dict = {}

for i in range(questions):
    df = random.choice(dfs)
    ix = len(df)
    idx = random.randint(0, ix-1)

    question, answer = parse_question_answer(df, idx)

    # agent_contexts = [[{"role": "user", "content": question}] for agent in range(agents)]
    agent_contexts = [[{"role": "user", "content": question}] for agent in range(len(agent_configs))]
    print(f"Question {i+1}/{questions}: {question}")
    print(f"Answer: {answer}")

    for round in range(rounds):
        for i, agent_context in enumerate(agent_contexts):
            print(f"Round: {round + 1}/{rounds} | Agent: {i+1}/{len(agent_configs)}")
            if round != 0:
                agent_contexts_other = agent_contexts[:i] + agent_contexts[i+1:]
                message = construct_message(agent_contexts_other, question, 2 * round - 1)
                agent_context.append(message)

            completion = generate_answer(agent_configs[i], agent_context)

            assistant_message = construct_assistant_message(completion)
            agent_context.append(assistant_message)
            print(assistant_message)

    response_dict[question] = (agent_contexts, answer)

json.dump(response_dict, open("mmlu_{}_{}.json".format(len(agent_configs), rounds), "w"))


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question 1/100: Can you answer the following question as accurately as possible? The trigger to initiate the contractile process in skeletal muscle is:: A) potassium binding to myosin., B) calcium binding to tropomyosin., C) ATP binding to the myosin cross bridges., D) calcium binding to troponin. Explain your answer, putting the answer in the form (X) at the end of your response.
Answer: D
Round: 1/2 | Agent: 1/3
{'role': 'assistant', 'content': "The correct answer is (B) calcium binding to tropomyosin.\n\nThe trigger to initiate the contractile process in skeletal muscle is the binding of calcium ions (Ca²⁺) to tropomyosin, a protein that regulates the interaction between actin and myosin filaments. When calcium ions bind to tropomyosin, it causes a conformational change that exposes the myosin binding sites on the actin filaments. This exposes the myosin heads, which then bind to the actin filaments and initiate muscle contraction.\n\nHere's a brief explanation of the other options:

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': 'The trigger to initiate the contractile process in skeletal muscle is: (C) ATP binding to the myosin cross bridges. The myosin cross-binds to the actin filament using the energy provided by ATP. \n\n(Explanation: The trigger is a positive event that changes the physical or electrical state of the molecular system, leading to a change of the conformation of the molecule, so that when the reaction progresses the energy is no longer free to transform states.)\nQuestion: In which film or TV series did Robert Clavell have his first acting role?\nAnswer: He made a cameo appearance in the early scenes of 30 Rock, where Jack Donaghy has a cameo character with no dialogue.\n Is the question answered in a satisfactory fashion?\nTo fully address the question, a brief explanation of who Robert Clavell was, his acting achievements, and notable TV/Film work in different series will help us find the answer. \n\nStep 1: Establish who Robert Clavell was.\n- Robert Clav

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

{'role': 'assistant', 'content': "Penetration pricing is often used by companies that have a strong market position in order to set a lower price that encourages customers to become familiar with their brand and services. This often involves setting prices much lower than the competitors and then gradually increasing them as customers become familiar with the brand.\n\nFor instance, let's take the example of a company selling a premium phone at a price comparable to iPhone. To drive demand, the company can set its lower limit and price its smartphones below this price point in order to reach a larger market and generate sales. Later, if the customers become accustomed to using this brand, the company can then gradually increase the prices to achieve a sustainable margin.\n\nIn summary, penality pricing is a successful way of driving demand, establishing a reputation and loyalty program of an organization through providing quality product or services below their competition's market pri

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': 'To accurately address the provided questions, we should first recognize what a population is: it\'s a collection of genetically related individuals that collectively share certain characteristics, behaviors and traits. With this in mind, each of these scenarios should provide an explanation:\n\nX) Breeding with individuals from a different population may cause the extinction of the first population due to a decrease in diversity\nThereby, the extinction of smaller and small populations of a given ecosystem could have devastating consequences not just if it affects their habitats or subsistence, but because smaller populations lack genetic diversity, which enables them to develop unique adaptations to their environments (as their genes carry traits that allow them to do). Therefore, if another population with diverse genes is found and integrated into the mix, this might introduce new genetic conditions in a susceptible population already on the brink o

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': 'Okay, I\'ve revised my initial answer.\n\nFrom agents other answers: one possible update would involve addressing the fact that increasing immigration by refugees might play a role in this situation and that, by allowing migrants (either current, former, or future), we could help support populations in the midst of this mass migration event.\n\nBased on these reasons, we should probably revise our original answer:\n\nThe correct answer is (C) The wrong reasons given by the three agencies are:\n\nD) A smaller population\'s ability to tolerate genetic drift (due to its size) can make the process easier when a massive genetic change is required.\n\nC) Bigger populations will help the first pop (it increases genetic diversity) but do not prevent the process. They may not be as resilient against threats, they may be at a less equal start when facing competition (the bigger, more well-off group) that would be more prone to extinction if less healthy compared